<a href="https://colab.research.google.com/github/yoneken1/colab_pytorch_detection/blob/master/Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

smooth_l1_loss
> original:https://github.com/chenyuntc/simple-faster-rcnn-pytorch

In [0]:
import torch
from torch import nn

def smooth_l1_loss(x, t, in_weight, sigma):
    sigma2 = sigma ** 2
    diff = in_weight * (x - t)
    abs_diff = diff.abs()
    flag = (abs_diff.data < (1. / sigma2)).float().detach()
    y = (flag * (sigma2 / 2.) * (diff ** 2) +
         (1 - flag) * (abs_diff - 0.5 / sigma2))
    return y
  
def test():
  x = torch.randn([16,4])
  t = torch.randn([16,4])
  in_weight = torch.zeros((x.size(0),1)).float()
  in_weight[0] = 1.
  
  print(smooth_l1_loss(x,t,in_weight,1.))
  
test()



tensor([[0.0137, 1.2626, 0.5101, 1.5306],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])


In [0]:
class DetectionLoss(nn.Module):
  
  def __init__(self, class_agnostic=False):
    super(DetectionLoss, self).__init__()
    self.loc_sigma = 1.
    self.class_agnostic = class_agnostic
    self.cls_loss = torch.nn.CrossEntropyLoss(ignore_index=-1,reduction='sum')
    
  def forward(self,pred_loc,pred_cls,target_loc,target_cls, loc_weight):
    '''Make training target
      Args:
        pred_loc: (tensor)bounding boxes, sized [N,4].
        pred_cls: (tensor) bounding boxes labels, sized [N,num_classes+1]. 0 = background
        target_loc: (tensor) boxes, sized [N,4].
        target_cls: (long tensor) labels, sized [N]. 0 = background, -1 = ignore
        loc_weight: (tensor) bounding boxes, sized [N,1].
      Return:
        (tensor) loc_loss, sized [N].
        (tensor) cls_loss, sized [N].
        
  
    '''
    if not self.class_agnostic:
      use_ids = (loc_weight > 0).view(-1)
      loc_mask = torch.zeros((pred_loc.size(0),int(pred_loc.size(1)/4),4)).byte().to(device)
      loc_mask[use_ids,target_cls[use_ids]-1] = 1
      loc_mask[use_ids==0,0] = 1
      loc_mask = loc_mask.view(pred_loc.size(0),-1)
      pred_loc = pred_loc[loc_mask].view(pred_loc.size(0),4)
    
    loc_loss = smooth_l1_loss(pred_loc,target_loc,loc_weight,self.loc_sigma).sum()
    cls_loss = self.cls_loss(pred_cls,target_cls)
    return loc_loss,cls_loss

device=torch.device('cuda') 
  
def test():
  loss = DetectionLoss().to(device)
  pred_loc = torch.randn((16,4)).to(device)
  target_loc = torch.randn((16,4)).to(device)
  loc_weight = torch.zeros((pred_loc.size(0),1)).float().to(device)
  loc_weight[0] = 1.
  pred_cls = torch.randn((16,8)).float().to(device)
  target_cls = torch.tensor([0,1,2,3,4,5,6,7,-1,-1,-1,-1,-1,-1,-1,-1]).long().to(device)
  
  print(loss(pred_loc,pred_cls,target_loc,target_cls, loc_weight))
  
test()

(tensor([0.8253, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0'), tensor([1.4757, 1.1150, 3.1874, 1.1488, 3.1614, 3.0009, 2.6451, 2.0399, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0'))


In [0]:
def calc_detection_loss(loss, pred_loc, pred_cls, target_loc, target_cls, pos_ids, neg_ids,
                       difficult_ids=None):
  
  #create loc weight. 1:positive 0:others
  loc_weight = torch.zeros((pred_loc.size(0),1)).float().to(device)
  if(pos_ids.size(0)>0):
    loc_weight[pos_ids] = 1.
  if( (difficult_ids is not None) & (difficult_ids.size(0) > 0)):
    loc_weight[difficult_ids] *= 0.01
  
  #create target cls for loss 0:negative, -1:ignore, 1 to num_classes:positive class label + 1
  target_cls_for_loss = torch.Tensor((target_cls.size(0))).long().fill_(-1.).to(device)
  if(neg_ids.size(0)>0):
    target_cls_for_loss[neg_ids] = 0
  if(pos_ids.size(0)>0):
    target_cls_for_loss[pos_ids] = target_cls[pos_ids] + 1
    
  loc_loss, cls_loss = loss(pred_loc,pred_cls,target_loc,target_cls_for_loss, loc_weight)
  
  return loc_loss, cls_loss
   
#device = torch.device('cpu')
  
def test():
  
  loss = DetectionLoss().to(device)
  pred_loc = torch.randn((16,4)).to(device)
  pred_cls = torch.randn((16,9)).float().to(device)
  target_loc = torch.randn((16,4)).to(device)
  target_cls = torch.tensor([0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7]).long().to(device)
  pos_ids = torch.tensor([0,1,2]).to(device)
  neg_ids = torch.tensor([3,4,5]).to(device)
  loc_loss, cls_loss = calc_detection_loss(loss, pred_loc, pred_cls, target_loc, target_cls, pos_ids, neg_ids)  
  
  print(loc_loss)
  print(cls_loss)

test()
  
  